## <div style="text-align: right"> Mario Stanke, March 3rd, 2021 </div>

# Learning to Find Homologous Genome Regions

## 1. Task
### Input
   - A set $G$ of genome regions from $s$ genomes.  
   Each region $\alpha \in \{A,C,G,T\}^{\ell}$ is of limited length, say $\ell \le 10^4$, e.g. from tiling longer chromosomes.
   - Repeat locations

### Output
   1. **Homology clusters** $\{S\in G :  \varphi(S) = j\}$ for nonempty slots $j\in \{0,\ldots, m-1\}$


### Use Cases
   - Initial step in non-progressive multiple genome aligmnment
   - Sequence clustering

### Data Distribution
   - number of species $s\approx 100$, total genome length $n\approx 10^9$
   - one-to-one-to-one homology dominates
      - The number of sequence clusters wrt homology is in the order of the number of regions per genome,  
        too big for $k$-means clustering.
      - The average size of a cluster is in the order of $s$.
            
### Challenge
   - no pairwise comparison of sequences (too slow)
   - should preferrably scale linear in number of genomes $s$

In [1]:
import itertools
import numpy as np
import sys
from functools import cmp_to_key
import tensorflow as tf

dna_alphabet = "ACGT"
complements = "TGCA"
rctbl = str.maketrans(dna_alphabet, complements)
dna_alphabet_size = len(dna_alphabet)

## 3. $k$-mers
### Protein Sequences
Define alphabets and six_frame_translation.

In [2]:
codon_len = 3
codon_alphabet_size = dna_alphabet_size ** codon_len # 64
genetic_code = { # translation table 1 of NCBI
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M', 
    'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T', 
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K', 
    'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L', 
    'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P', 
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q', 
    'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R', 
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V', 
    'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A', 
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E', 
    'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G', 
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S', 
    'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L', 
    'TAC':'Y', 'TAT':'Y', 'TAA':'*', 'TAG':'*', 
    'TGC':'C', 'TGT':'C', 'TGA':'*', 'TGG':'W', 
}

aa_alphabet = list(set(genetic_code.values()))
print(aa_alphabet)

def six_frame_translation(S):
    """ return all 6 conceptually translated protein sequences """
    T = []
    for seq in (S, S[::-1].translate(rctbl)): # forward, reverse-complement sequence
        for f in range(3): # frame
            prot = ""
            for i in range(f, len(S) - codon_len + 1, codon_len):
                prot += genetic_code[seq[i:i+codon_len]]
            T.append(prot)
    return T


['K', 'N', 'R', 'C', 'Q', 'M', 'P', 'Y', 'T', '*', 'G', 'I', 'L', 'D', 'H', 'V', 'S', 'A', 'F', 'W', 'E']


In [3]:
six_frame_translation("ATGACAT")

['MT', '*H', 'D', 'MS', 'CH', 'V']

In [4]:
nuc_idx = dict((c,i) for i,c in enumerate(dna_alphabet))
aa_idx = dict((c,i) for i,c in enumerate(aa_alphabet))

def to_idx(seq, idx): # used later to one-hot encode
    return np.array(list(idx[c] for c in seq))

In [5]:
# example conversion of character to index sequence
to_idx("AACTG", nuc_idx)

array([0, 0, 1, 3, 2])

### 3.1 Compute $k$-mer Counts for Several Values of $k$ (DNA and 6-Frame-Translation)

In [6]:
nuc_ks = [5, 7] # values of k considered for DNA
aa_ks =  [3] # values of k considered for proteins

def kmer_indices(ks, alphabet):
    C = {}
    for k in ks:
        tuples = itertools.product(*[alphabet for i in range(k)])
        tuples = [''.join(c) for c in tuples]
        C[k] = {c:i for (i,c) in enumerate(tuples)}
    return C 

Cnuc  = kmer_indices(nuc_ks, dna_alphabet)
Cprot = kmer_indices(aa_ks, aa_alphabet)

In [7]:
print ("Indices of nucleotide and amino acid tuples")
for (ks, C) in ((nuc_ks, Cnuc), (aa_ks, Cprot)):
    for k in ks:
        print (list(C[k].items())[0:8])

Indices of nucleotide and amino acid tuples
[('AAAAA', 0), ('AAAAC', 1), ('AAAAG', 2), ('AAAAT', 3), ('AAACA', 4), ('AAACC', 5), ('AAACG', 6), ('AAACT', 7)]
[('AAAAAAA', 0), ('AAAAAAC', 1), ('AAAAAAG', 2), ('AAAAAAT', 3), ('AAAAACA', 4), ('AAAAACC', 5), ('AAAAACG', 6), ('AAAAACT', 7)]
[('KKK', 0), ('KKN', 1), ('KKR', 2), ('KKC', 3), ('KKQ', 4), ('KKM', 5), ('KKP', 6), ('KKY', 7)]


In [8]:
def kmer_counts(S, C, alphabet, k, tile_size = 1):
    counts = np.zeros(len(alphabet)**k)
    for seq in S:
        for i in range(0, len(seq) - k, tile_size):
           idx = C[k][seq[i:i+k]]
           counts[idx] += 1
    return counts

In [9]:
# define two similar example strings to test their comparison
α = "AAATGTGTCTACTGTGTCTGTGAGTGATAAAATATTTTAGGATGAGAGGCGCCGATCGCAGCGACAGCGCATCA"
# delete 2 chars from α, repace 2 chars, add 4 chars 
β = "AAATGAAATGTCTACTGTGTCTGTGAGTGATAAAATATTTTAGGATGAGAGGCGCCGATCGCAGCGACAGCGCATCA"

### Obtain Feature Vector for Sequence

In [10]:
def kmer_features(S):
    """ Obtain all counts of all k-mers (DNA, AA) concatenated."""
    T = six_frame_translation(S)
    featurelist = []
    for (ks, C, alphabet) in ((nuc_ks, Cnuc, dna_alphabet),
                              (aa_ks, Cprot, aa_alphabet)):
        for k in ks:
            featurelist.append(kmer_counts([S], C, alphabet, k))
    return np.concatenate(featurelist)

In [11]:
x_alpha = kmer_features(α)
x_beta = kmer_features(β)
x_alpha

array([0., 0., 0., ..., 0., 0., 0.])

In [12]:
p = x_alpha.shape[0]
print ("number of k-mer features: ", p, # can be expected to be large
       ", thereof non-zero: ", np.sum(x_alpha > 0))

number of k-mer features:  26669 , thereof non-zero:  169


#### Make features specific to input regions $G$
Long $k$-mers avoid noise. But there are too many. 
**Solution: 2 passes**
   1. First determine set of all $k$-mers (multiple $k$'s). [Jellyfish](https://github.com/gmarcais/Jellyfish)
   2. Fix feature set, e.g. a (random but) fixed subset of all occuring $k$-mers.
   3. Loop over $G$ again to compute features.

## 4. Feature Ideas
### 4.1 Profiles
 1. Can detect approximate patterns, rather than exact $k$-mers
 2. Have trainable parameters

Let $$p = (p_{u,v,w}) \in [0,1]^{U \times 4 \times k}$$
be a collection of $U$ DNA **profiles**, each of length $k$.
Let $$E = (e_{v,t}) \in \{0,1\}^{4 \times \ell}$$ be a **one-hot encoded input DNA sequence**.
Define learnable **profile features** $x = (x_{u}) \in \mathbb{R}^U$ by
$$ x_u = \max_{t=1}^{\ell-k+1} \sum_{w=1}^k \sum_{v=1}^4 e_{v,t+w} \ln p_{u,v,w} ,$$

which is similar -- and an alternative to -- a **one dimensional convolutional neural network (1-dim CNN)** with a max pooling.

### 4.2 Estimated Ancestral Patterns
The same features cannot be expected to work at **different evolutionary distances**. However, pairwise sequence distances are generally diverse when considering multiple genomes simultaneously or retraining for specific pairwise distances is not done.

<img src="tree.jpg" alt="hashing" style="width: 300px;"/>

Let $\tau=(\tau_1,\tau_2,\ldots, \tau_r)$ be a few evolutionary distances that represent the expected pairwise distances sufficiently.
Let $Q\in\mathbb{R}_+^{20\times 20}$ be an amino acid rate matrix of a time-reversible continuous-time Markov chain (CTMC) $X(t)$.

Let again $E = (e_{v,t}) \in \{0,1\}^{\ell \times 20}$ be the one-hot encoding of an input protein sequence $S$ and let 
$z = [0,1]^{r \times \ell \times 20}$, where
$$ z_{r,t,j} := P(X(\tau_r) = j \,|\, X(0) = S[t]).$$

In [13]:
# Jukes-Cantor amino acid model for demo only
s = len(aa_alphabet)

Q = 0.01 *np.ones((s, s)) - 0.2 * np.eye(s)
# the rows sum up to 0

τ = np.array([0.01, 0.1, 1.]) # considered evolutionary times

S = "KALK" # example amino acid sequence

In [14]:
def dna_one_hot(seq):
    return np.eye(len(dna_alphabet))[to_idx(seq, nuc_idx)]

def aa_one_hot(seq):
    return np.eye(len(aa_alphabet))[to_idx(seq, aa_idx)]

dna_one_hot("ATG")

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [15]:
E = aa_one_hot(S)
print(E.shape) # (4,21)
print(E)

tauQ = τ[..., None, None] * Q[None, ...]
tauQ.shape # (3, 21, 21)

P = tf.linalg.expm(tauQ) # P[r,i,j] = P(X(tau_r) = j | X(0) = i)

ancprobs = tf.einsum("ti,rij -> rtj", E, P) # Einstein sum, compute all ancestral character probs simultaneously
ancprobs = ancprobs.numpy()

print(ancprobs.shape) # (3, 4, 21)
with np.printoptions(precision = 4, suppress = True):
    print("ancestral profiles\n", ancprobs[:,:,0:8])

(4, 21)
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
(3, 4, 21)
ancestral profiles
 [[[0.9981 0.0001 0.0001 0.0001 0.0001 0.0001 0.0001 0.0001]
  [0.0001 0.0001 0.0001 0.0001 0.0001 0.0001 0.0001 0.0001]
  [0.0001 0.0001 0.0001 0.0001 0.0001 0.0001 0.0001 0.0001]
  [0.9981 0.0001 0.0001 0.0001 0.0001 0.0001 0.0001 0.0001]]

 [[0.9812 0.001  0.001  0.001  0.001  0.001  0.001  0.001 ]
  [0.001  0.001  0.001  0.001  0.001  0.001  0.001  0.001 ]
  [0.001  0.001  0.001  0.001  0.001  0.001  0.001  0.001 ]
  [0.9812 0.001  0.001  0.001  0.001  0.001  0.001  0.001 ]]

 [[0.8278 0.0091 0.0091 0.0091 0.0091 0.0091 0.0091 0.0091]
  [0.0091 0.0091 0.0091 0.0091 0.0091 0.0091 0.0091 0.0091]
  [0.0091 0.0091 0.0091 0.0091 0.0091 0.0091 0.0091 0.0091]
  [0.8278 0.0091 0.0091 0.0091 0

### 4.3 Iterated Sum Signatures
Contigous patterns like above cannot capture
  1. similar sequences with indels
  2. shared sequences of patterns, such as peptide $\alpha_1$ is followed by peptide $\alpha_2$ is followed by peptide $\alpha_3$ where the peptides can be in different exons.
  
Let $S$ be a sequence of length $\ell$ and 
$p=(p^1, p^2,\ldots, p^r)$ be a sequence of $r$ profiles of lengths $k_1,\ldots, k_r$ (fixed $k$-mers can be a special case).

<img src="ISS.jpg" alt="hashing" style="width: 700px;"/>

$$ \text{ISS} = \sum_{1\le i_1 < i_1+k_1 < i_2 < i_2 + k_2 < \cdots < i_k \le \ell} \prod_{j=1}^r g(i_j) \lambda(\ell_j)$$

Here, $g(i_j)$ is the probability of $S[i_j:i_j+k_j]$ under profile $p^j$ and
$\ell_j:= i_j -i_{j-1} + k_{j-1}\ge 0$ is the length of the gap between pattern occurrences $j-1$ and $j$.
$\lambda$ is a gap penalty function.

ISS can be computed with dynamic programming (DP) in time $O(\ell k)$, where $k=k_1+ \cdots + k_r$ is the total pattern length, uder conditions on $\lambda$, e.g. $\ln\lambda$ is affine linear.
